### Extract top-N scoring clips per Site ID and SN. 

Then we will sort the predictions by score using the column 'Confidence' and select the top-N clip per site and survey night. Then save this selection (including the column begin Path, confidence and offset) to use it in the next step that will be extract the clips from long recorings using Audio from OpenSoundscape.

In [1]:
import os
import pandas as pd
from pathlib import Path
from opensoundscape import Audio
from tqdm.autonotebook import tqdm

/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/ml/cnn.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [17]:
main_dir = "/mnt/d/night_recordings_analysis/Mottled Owl_song/"
top_n = 1  # Set the number of top clips per Site_ID and SN
clip_duration = 3  # seconds

In [18]:
# STEP 1: Load and rename columns
predictions = pd.read_csv(os.path.join(main_dir, "motowl_song_SelectionTable.txt"), sep='\t')

predictions.rename(columns={
    'Begin Path': 'file_path',
    'File Offset (s)': 'offset',
    'Common Name': 'class',
    'Species Code': 'class_code',
    'Confidence': 'score',
}, inplace=True)

Sort predictions by score and Site ID, SN & Score and select the top N clips 

In [19]:
# STEP 2: Group and select top-N clips per Site_ID and SN

predictions_sorted = predictions.sort_values(by='score', ascending=False)
predictions_sorted.head()

,Selection,class,class_code,score,file_path,offset,Site_ID,SN
46298,46299,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.9978,/mnt/d/Disco3_Backup/night_recordings/M5-ARU21...,99.0,M5,8
46388,46389,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.9954,/mnt/d/Disco3_Backup/night_recordings/M5-ARU21...,18.0,M5,8
46377,46378,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.9937,/mnt/d/Disco3_Backup/night_recordings/M5-ARU21...,105.0,M5,8
46251,46252,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.9925,/mnt/d/Disco3_Backup/night_recordings/M5-ARU21...,63.0,M5,8
46406,46407,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.9921,/mnt/d/Disco3_Backup/night_recordings/M5-ARU21...,75.0,M5,8


In [20]:
top_clips = predictions_sorted.groupby(['Site_ID', 'SN']).head(top_n).reset_index(drop=True)
top_clips.head()

,Selection,class,class_code,score,file_path,offset,Site_ID,SN
0,46299,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.9978,/mnt/d/Disco3_Backup/night_recordings/M5-ARU21...,99.0,M5,8
1,48072,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.9730,/mnt/d/Disco3_Backup/night_recordings/M5-ARU21...,84.0,M5,14
2,101399,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.2696,/mnt/d/Disco4_Backup/night_recordings/M27-ARU3...,54.0,M27,13
3,28323,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.2242,/mnt/d/Disco3_Backup/night_recordings/CH9-ARU1...,48.0,CH9,15
4,84531,Mottled Owl_song,Ciccaba virgata_Mottled Owl_song,0.2161,/mnt/d/Disco6_Backup/night_recordings/CH47-ARU...,30.0,CH47,4


Using the 'file_path' column & 'offset', **extract the clips from long recordings for validation**

In [21]:
# STEP 3: Extract and save clips
for idx, row in top_clips.iterrows():
    site_id = row['Site_ID']
    sn = row['SN']
    offset = int(row['offset'])
    score = row['score']
    full_audio_path = Path(row['file_path'])
    
    # Construct subdirectory and filename
    out_dir = Path(main_dir)
    #site_dir.mkdir(parents=True, exist_ok=True)

    # Format output filename
    output_filename = f"{score:.4f}_{offset}_{site_id}_SN{sn}.WAV".lower() 
    output_path = out_dir / output_filename
    
    try:
        # Load and extract clip
        audio = Audio.from_file(full_audio_path, offset=offset, duration=clip_duration)
        audio.save(output_path)
        print(f"Saved: {output_path}")
    except Exception as e:
        print(f"Error processing {full_audio_path} at offset {offset}: {e}")

print(f"\n Done! Clips saved to: {main_dir}")

Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.9978_99_m5_sn8.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.9730_84_m5_sn14.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.2696_54_m27_sn13.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.2242_48_ch9_sn15.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.2161_30_ch47_sn4.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.2151_51_ch3_sn4.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.2109_39_ch47_sn2.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.2099_102_m5_sn13.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.2039_9_ch47_sn6.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1978_114_ch47_sn11.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1795_117_m4_sn1.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1761_57_ch25_sn15.wav
Saved: /mnt/d/night_recordings_analysis/Mot

/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/audio.py:340: UserWarning: Audio object is shorter than requested duration: 2.0 sec instead of 3 sec
  warnings.warn(error_msg)


Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1465_117_p26_sn8.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1450_27_m43_sn8.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1425_0_ch8_sn12.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1407_81_m5_sn16.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1277_30_p4_sn7.wav


/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/audio.py:340: UserWarning: Audio object is shorter than requested duration: 2.0 sec instead of 3 sec
  warnings.warn(error_msg)


Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1188_117_m27_sn20.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1181_108_m7_sn3.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1156_96_p35_sn13.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1092_72_ch8_sn16.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1088_21_ch37_sn8.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1075_30_m4_sn3.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1053_21_ch19_sn3.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1036_6_p47_sn14.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.1012_90_ch38_sn3.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0998_117_ch9_sn3.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0984_0_ch8_sn17.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0930_69_m5_sn21.wav
Saved: /mnt/d/night_recordings_analysis/Mottl

...

Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_117_m33_sn1.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_90_p40_sn5.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_78_p7_sn6.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_117_m25_sn16.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_45_p25_sn13.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_0_p25_sn9.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_6_p25_sn2.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_42_p25_sn5.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_51_p25_sn6.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_30_ch34_sn10.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_114_ch34_sn14.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_39_m29_sn3.wav
Saved: /mnt/d/night_recordings_analysis/Mottle

/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/audio.py:340: UserWarning: Audio object is shorter than requested duration: 2.0 sec instead of 3 sec
  warnings.warn(error_msg)


Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_117_m34_sn24.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_114_ch16_sn15.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_117_p29_sn28.wav


/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/audio.py:340: UserWarning: Audio object is shorter than requested duration: 2.0 sec instead of 3 sec
  warnings.warn(error_msg)


Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_111_p22_sn12.wav
Saved: /mnt/d/night_recordings_analysis/Mottled Owl_song/0.0010_30_p20_sn11.wav

 Done! Clips saved to: /mnt/d/night_recordings_analysis/Mottled Owl_song/
